In [ ]:
include("_.jl")

In [ ]:
me_ge_ = OrderedDict()

In [ ]:
en = OnePiece.gene.read_ensembl(or = "mouse")

pr_ = en[!, "Gene type"] .== "protein_coding"

pr_ = sort(unique(skipmissing(en[pr_, "Gene name"])))

## Ensembl

In [ ]:
mo_hu = OnePiece.gene.map_with_mouse()

me_ge_["Ensembl"] = [pr for pr in pr_ if !haskey(mo_hu, pr)]

## Mouse Genome Informatics

In [ ]:
mg = OnePiece.table.read(joinpath(PAI, "hom_mousehumansequence.rpt.txt.gz"))

hu_ = mg[!, "Common Organism Name"] .== "human"

mg[hu_, "Symbol"] .= OnePiece.gene.rename(mg[hu_, "Symbol"])[1]

mo_hu = Dict()

for da in groupby(mg, "DB Class Key")

    sy_ = da[!, "Symbol"]

    mo_ = da[!, "Common Organism Name"] .== "mouse, laboratory"

    for mo in sy_[mo_], hu in sy_[.!mo_]

        mo_hu[mo] = hu

    end

end

println(length(mo_hu))

me_ge_["MGI"] = [pr for pr in pr_ if !haskey(mo_hu, pr)]

## Write

In [ ]:
write_gene_by_method(PAO, "mouse", me_ge_)